In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

Read CSV file

In [2]:
df = pd.read_csv('../csv/chess_games.csv')

Print 5 rows from dataframe

In [3]:
df.head()

,Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
0,Classical,eisaaaa,HAMID449,1-0,2016.06.30,22:00:01,1901,1896,11.0,-11.0,D10,Slav Defense,300+5,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1,Blitz,go4jas,Sergei1973,0-1,2016.06.30,22:00:01,1641,1627,-11.0,12.0,C20,King's Pawn Opening: 2.b3,300+0,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2,Blitz tournament,Evangelistaizac,kafune,1-0,2016.06.30,22:00:02,1647,1688,13.0,-13.0,B01,Scandinavian Defense: Mieses-Kotroc Variation,180+0,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....
3,Correspondence,Jvayne,Wsjvayne,1-0,2016.06.30,22:00:02,1706,1317,27.0,-25.0,A00,Van't Kruijs Opening,-,Normal,1. e3 Nf6 2. Bc4 d6 3. e4 e6 4. Nf3 Nxe4 5. Nd...
4,Blitz tournament,kyoday,BrettDale,0-1,2016.06.30,22:00:02,1945,1900,-14.0,13.0,B90,"Sicilian Defense: Najdorf, Lipnitsky Attack",180+0,Time forfeit,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...


Print unique values from column *Event*. As you can see there are a lot of similiar values. So I have to change some of them.

In [4]:
print(df['Event'].unique())

[' Classical ' ' Blitz ' ' Blitz tournament ' ' Correspondence '
 ' Classical tournament ' ' Bullet tournament ' ' Bullet '
 'Blitz tournament ' 'Bullet ' 'Classical ' 'Blitz ' 'Bullet tournament '
 'Classical tournament ' 'Correspondence ']


Print information of dataframe

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6256184 entries, 0 to 6256183
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Event            object 
 1   White            object 
 2   Black            object 
 3   Result           object 
 4   UTCDate          object 
 5   UTCTime          object 
 6   WhiteElo         int64  
 7   BlackElo         int64  
 8   WhiteRatingDiff  float64
 9   BlackRatingDiff  float64
 10  ECO              object 
 11  Opening          object 
 12  TimeControl      object 
 13  Termination      object 
 14  AN               object 
dtypes: float64(2), int64(2), object(11)
memory usage: 716.0+ MB


Number of unique values for every column

In [6]:
df.nunique()

Event                   14
White               118945
Black               115946
Result                   4
UTCDate                 32
UTCTime              86400
WhiteElo              2174
BlackElo              2181
WhiteRatingDiff       1177
BlackRatingDiff       1163
ECO                    493
Opening               2942
TimeControl            841
Termination              5
AN                 6188849
dtype: int64

Most common value from each column

In [7]:
round(df.describe(),2)

,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff
count,6256184.00,6256184.00,6251516.00,6251516.00
mean,1741.95,1740.39,0.58,-0.37
std,265.72,266.89,22.85,21.82
min,737.00,728.00,-595.00,-653.00
25%,1559.00,1557.00,-9.00,-10.00
50%,1740.00,1739.00,1.00,-1.00
75%,1919.00,1919.00,10.00,9.00
max,3110.00,3108.00,673.00,664.00


I'm changing types from *object* into *string*

In [8]:
df['Event'] = df['Event'].astype('string')
df['White'] = df['White'].astype('string')
df['Black'] = df['Black'].astype('string')
df['Result'] = df['Result'].astype('string')
df['UTCDate'] = df['UTCDate'].astype('string')
df['UTCTime'] = df['UTCTime'].astype('string')
df['ECO'] = df['ECO'].astype('string')
df['Opening'] = df['Opening'].astype('string')
df['TimeControl'] = df['TimeControl'].astype('string')
df['Termination'] = df['Termination'].astype('string')
df['AN'] = df['AN'].astype('string')


Drop columns *UTCDate* and *UTCTime*, because I don't need them

In [9]:
df_drop = df.drop(['UTCDate', 'UTCTime'], axis=1)

Drop all NaN values from dataframe

In [10]:
df_drop = df_drop.dropna()

Changing similiar values into one. As you can see, there are values with space before and after value, so I merge them into one value.

In [11]:
df_drop['Event'] = df_drop['Event'].replace([' Classical ', ' Classical tournament ', 'Classical ', 'Classical tournament '], 'Classical')
df_drop['Event'] = df_drop['Event'].replace([' Blitz ', ' Blitz tournament ', 'Blitz ', 'Blitz tournament '], 'Blitz')
df_drop['Event'] = df_drop['Event'].replace([' Bullet ', ' Bullet tournament ', 'Bullet ', 'Bullet tournament '], 'Bullet')
df_drop['Event'] = df_drop['Event'].replace([' Correspondence ', 'Correspondence '], 'Correspondence')

Replacing result values into numeric, because it will help in the future.

1 - means White wins
2 - means Black wins
3 - means it's a draw

Also, I drop all values with result "*"

In [12]:
df_drop['Result'] = df_drop['Result'].replace(['1-0'], 1)
df_drop['Result'] = df_drop['Result'].replace(['0-1'], 2)
df_drop['Result'] = df_drop['Result'].replace(['1/2-1/2'], 3)
df_drop = df_drop[df_drop['Result'] != '*']

Create new dataframes and drop rows where termination were *time forfeit* and it was a draw.

In [13]:
df_new = df_drop[['Event','Result','WhiteElo','BlackElo','WhiteRatingDiff','BlackRatingDiff','ECO','Opening','TimeControl','Termination']]
df_new = df_new.drop(df_new[(df_new['Termination'] == 'Time forfeit') & (df_new['Result'] == 3)].index)
df_filter = df_new

Drop rows where termination were *rules infraction* or *abandoned* because I will focus only on games ended by checkmate or time forfeit

In [14]:
df_filter = df_filter[df_filter['Termination'] != "Rules infraction"]
df_filter = df_filter[df_filter['Termination'] != "Abandoned"]

Split strings in *TimeControl* column. Only values before *+* will stay.

In [15]:
df_filter['TimeControl'] = df_filter['TimeControl'].str.split('+').str[0]

Set 0 for rows where *TimeControl* was *-* (for all Correspondence games)

In [16]:
df_filter['TimeControl'] = df_filter['TimeControl'].replace('-', 0)

Change type on int64

In [17]:
df_filter['TimeControl'] = df_filter['TimeControl'].astype('int64')

I'm using LabelEncoder to change values from string columns into numeric values

In [18]:
labelencoder = LabelEncoder()

In [19]:
df_filter['Event_enc'] = labelencoder.fit_transform(df_filter['Event'])
df_filter['ECO_enc'] = labelencoder.fit_transform(df_filter['ECO'])
df_filter['Termination_enc'] = labelencoder.fit_transform(df_filter['Termination'])
df_filter['TimeControl_enc'] = labelencoder.fit_transform(df_filter['TimeControl'])

In [20]:
df_enc = df_filter[['Result','WhiteElo','BlackElo','Event_enc','ECO_enc','Termination_enc','TimeControl_enc']]

In [21]:
df_enc['Result'] = df_enc['Result'].astype('int64')

/var/folders/4v/xsywxnxn2m9dymh3jjbqsvym0000gn/T/ipykernel_14969/2691848660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enc['Result'] = df_enc['Result'].astype('int64')


In [22]:
df_enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6186502 entries, 0 to 6256183
Data columns (total 7 columns):
 #   Column           Dtype
---  ------           -----
 0   Result           int64
 1   WhiteElo         int64
 2   BlackElo         int64
 3   Event_enc        int64
 4   ECO_enc          int64
 5   Termination_enc  int64
 6   TimeControl_enc  int64
dtypes: int64(7)
memory usage: 377.6 MB


Create new dataframes where every frame contains games from each type (format) chess game. For example, in *df_blitz* will be only rows where Event (format) will be *Blitz*.

In [23]:
df_blitz = df_enc[df_enc['Event_enc'] == 0]
df_bullet = df_enc[df_enc['Event_enc'] == 1]
df_classical = df_enc[df_enc['Event_enc'] == 2]
df_correspondence = df_enc[df_enc['Event_enc'] == 3]

Print shapes of dataframes (rows, columns)

In [24]:
df_blitz.shape

(2780077, 7)

In [25]:
df_bullet.shape

(1719154, 7)

In [26]:
df_classical.shape

(1665069, 7)

In [27]:
df_correspondence.shape

(22202, 7)

Save dataframes into new CSV files

In [28]:
df_enc.to_csv('../csv/final.csv', index = False)
df_blitz.to_csv('../csv/final_blitz.csv', index = False)
df_bullet.to_csv('../csv/final_bullet.csv', index = False)
df_classical.to_csv('../csv/final_classical.csv', index = False)
df_correspondence.to_csv('../csv/final_correspondence.csv', index = False)
df_filter.to_csv('../csv/final_eda.csv', index = False)